---
### autograd 설명
- .require_grad : tracking all operations
- .backward() : grad 계산
- .grad : tensor에서 계산된 grad를 accumulate (보통 accumulate하지 않고, grad 초기화 후 사용)
<br><br>
- .detach() : compuation 시 tracking에서 제외 (메모리 save)
- with torch.no_grad() : .requires_grad=True를 하더라도 해당 구문안에서는 grad를 계산/사용하지 않음
<br><br>
- Function <-> Tensor : 모든 tensor(UDF제외)는 .grad_fn을 가질 수 있음(Function에서 refer)
---

In [11]:
import torch
import numpy as np

In [5]:
x = torch.ones(2,2, requires_grad=True)
print(x)

tensor([[1., 1.],
        [1., 1.]], requires_grad=True)


In [6]:
print(x.grad)

None


In [7]:
print(x.grad_fn)

None


In [10]:
y = x + 2 ## x에대가 2를 더하는 sum operation 적용 -> grad_fn 생성
print(y)
print(y.grad_fn) # tensor operation으로 생성되었기에 grad_fn이 존재

tensor([[3., 3.],
        [3., 3.]], grad_fn=<AddBackward>)


In [16]:
# *(multiplication) : position-wise mul (shape이 변하지 않음)
# dot product : inner-product라 생각하면 됨 (shape이 변함, 일반적으로 생각하는 matrix 곱)
a = np.array([[3,3]])
print(a, a.shape)

print(a.dot(a.transpose()))
print(a*a)

[[3 3]] (1, 2)
[[18]]
[[9 9]]


In [21]:
print(y, y.shape)
z = y*y*3
print(z)
out = z.mean()
print(out)
print(out.grad_fn)

tensor([[3., 3.],
        [3., 3.]], grad_fn=<AddBackward>) torch.Size([2, 2])
tensor([[27., 27.],
        [27., 27.]], grad_fn=<MulBackward>)
tensor(27., grad_fn=<MeanBackward1>)


In [29]:
# change .requires_grad
a = torch.rand(2,2)
print(a)
a = ((a*3)/(a-1))
print(a)

print(a.requires_grad)
a.requires_grad_(True)
print(a.requires_grad)

tensor([[0.4449, 0.9204],
        [0.4554, 0.1537]])
tensor([[ -2.4042, -34.7013],
        [ -2.5085,  -0.5446]])
False
True


In [30]:
b = (a*a).sum()
print(b.requires_grad)
print(b.grad_fn)

True


### Gradient

In [47]:
# out is single scalar -> out.backward = out.backward(torch.tensor(1))
print(out)
print(out.size)

x = torch.randn(3, requires_grad=True)
print(x.grad)
y = x*2
print(y, y.grad, y.grad_fn)
print(y.data.norm()) # y의 L2 norm(Euclidean)

tensor(27., grad_fn=<MeanBackward1>)
<built-in method size of Tensor object at 0x7f4416336708>
None
tensor([ 1.4342, -0.8942, -1.3027], grad_fn=<MulBackward>) None <MulBackward object at 0x7f4416357320>
tensor(2.1339)


In [48]:
i = 0
while y.data.norm() < 1000:
    i += 1
    y = y*2
print(y, i)

tensor([ 734.3131, -457.8081, -666.9732], grad_fn=<MulBackward>) 9


In [49]:
grads = torch.tensor([0.01, 1.0, 0.0001], dtype = torch.float)
y.backward(grads)

print(x)
print(y)
print(x.grad)

tensor([ 0.7171, -0.4471, -0.6513], requires_grad=True)
tensor([ 734.3131, -457.8081, -666.9732], grad_fn=<MulBackward>)
tensor([  10.2400, 1024.0000,    0.1024])


In [52]:
print(x.requires_grad)
print((x**2).requires_grad)

with torch.no_grad():
    print((x**2).requires_grad)

True
True
False
